### Домашнее задание к уроку 4

In [94]:
import matplotlib.pyplot as plt
import random

from matplotlib.colors import ListedColormap
from sklearn.datasets import make_classification, make_circles, make_regression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

import time

import warnings
warnings.filterwarnings('ignore')

In [67]:
# Реализуем класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [89]:
# И класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
#         Лазарева (
# закомментировано
#         # подсчет количества объектов разных классов
#         classes = {}  # сформируем словарь "класс: количество объектов"
#         for label in self.labels:
#             if label not in classes:
#                 classes[label] = 0
#             classes[label] += 1
            
#         # найдем класс, количество объектов которого будет максимальным в этом листе и вернем его    
#         prediction = max(classes, key=classes.get)

# добавлено
        prediction = self.labels.mean()
#         Лазарева )
        return prediction   

In [69]:
# Расчет критерия Джини

def gini(labels):
    #  подсчет количества объектов разных классов
    classes = {}
    for label in labels:
        if label not in classes:
            classes[label] = 0
        classes[label] += 1
    
    #  расчет критерия
    impurity = 1
    for label in classes:
        p = classes[label] / len(labels)
        impurity -= p ** 2
        
    return impurity

In [70]:
def mse(targets):
    return np.mean((targets - targets.mean())**2)

In [71]:
# Расчет прироста

def gain(left_labels, right_labels, root_criterion, criterion):

    # доля выборки, ушедшая в левое поддерево
    p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])
    
    return root_criterion - p * criterion(left_labels) - (1 - p) * criterion(right_labels)

In [72]:
# Разбиение датасета в узле

def split(data, labels, column_index, t):
    
    left = np.where(data[:, column_index] <= t)
    right = np.where(data[:, column_index] > t)
        
    true_data = data[left]
    false_data = data[right]
    
    true_labels = labels[left]
    false_labels = labels[right]
        
    return true_data, false_data, true_labels, false_labels

In [73]:
# Нахождение наилучшего разбиения

def find_best_split(data, labels):
    
    #  обозначим минимальное количество объектов в узле
    min_samples_leaf = 3
    
#     Лазарева (
#     Закомментировано
#     root_gini = gini(labels)

#     Добавлено
    root_mse = mse(labels)
#     Лазарева )

    best_gain = 0
    best_t = None
    best_index = None
    
    n_features = data.shape[1]
    
    for index in range(n_features):
        # будем проверять только уникальные значения признака, исключая повторения
        t_values = np.unique(data[:, index])
        
        for t in t_values:
            true_data, false_data, true_labels, false_labels = split(data, labels, index, t)
            #  пропускаем разбиения, в которых в узле остается менее 5 объектов
            if len(true_data) < min_samples_leaf or len(false_data) < min_samples_leaf:
                continue
            
            #     Лазарева (
            #     Закомментировано
            #     current_gain = gain(true_labels, false_labels, root_gini, gini)
            #    Добавлено
            current_gain = gain(true_labels, false_labels, root_mse, mse)
            #    Лазарева )
            
            #  выбираем порог, на котором получается максимальный прирост качества
            if current_gain > best_gain:
                best_gain, best_t, best_index = current_gain, t, index

    return best_gain, best_t, best_index

In [83]:
# Построение дерева с помощью рекурсивной функции

def build_tree(data, labels):

    gain, t, index = find_best_split(data, labels)

    #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
    if gain == 0:
        return Leaf(data, labels)

    true_data, false_data, true_labels, false_labels = split(data, labels, index, t)

    # Рекурсивно строим два поддерева
    true_branch = build_tree(true_data, true_labels)

#     print(time.time(), true_branch)
    false_branch = build_tree(false_data, false_labels)
    
#     print(time.time(), false_branch)
    
    # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
    return Node(index, t, true_branch, false_branch)

In [84]:
def classify_object(obj, node):

    #  Останавливаем рекурсию, если достигли листа
    if isinstance(node, Leaf):
        answer = node.prediction
        return answer

    if obj[node.index] <= node.t:
        return classify_object(obj, node.true_branch)
    else:
        return classify_object(obj, node.false_branch)

In [85]:
def predict(data, tree):
    
    classes = []
    for obj in data:
        prediction = classify_object(obj, tree)
        classes.append(prediction)
    return classes

In [91]:
# Напечатаем ход нашего дерева
def print_tree(node, spacing=""):

    # Если лист, то выводим его прогноз
    if isinstance(node, Leaf):
        print(spacing + "Прогноз:", node.prediction)
        return

    # Выведем значение индекса и порога на этом узле
    print(spacing + 'Индекс', str(node.index), '<=', str(node.t))

    # Рекурсионный вызов функции на положительном поддереве
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Рекурсионный вызов функции на отрицательном поддереве
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

#### __1.__ Реализуйте дерево для задачи регрессии. Возьмите за основу дерево, реализованное в методичке, заменив механизм предсказания в листе на взятие среднего значения по выборке, и критерий Джини на дисперсию значений.

In [86]:
# сгенерируем данные
data, targets = make_regression(n_features=2, n_informative=2, random_state=5)

In [87]:
train_data, test_data, train_labels, test_labels = train_test_split(data, 
                                                                    targets, 
                                                                    test_size=0.3,
                                                                    random_state=1)

In [92]:
# Построим дерево по обучающей выборке
my_tree = build_tree(train_data, train_labels)
print_tree(my_tree)

Индекс 0 <= -0.10061434630710828
--> True:
  Индекс 0 <= -0.8568531547160899
  --> True:
    Индекс 0 <= -1.4219245490984462
    --> True:
      Прогноз: -133.16917969924597
    --> False:
      Прогноз: -95.7089797243071
  --> False:
    Индекс 0 <= -0.5732155560138283
    --> True:
      Индекс 1 <= -0.4075191652021827
      --> True:
        Индекс 1 <= -1.2640833431434955
        --> True:
          Прогноз: -68.44174399666052
        --> False:
          Прогноз: -57.92411034120212
      --> False:
        Прогноз: -36.70317083946181
    --> False:
      Индекс 1 <= -0.3058530211666308
      --> True:
        Прогноз: -29.105630694331246
      --> False:
        Индекс 1 <= 0.18760322583703548
        --> True:
          Индекс 1 <= 0.0032888429341100755
          --> True:
            Прогноз: -10.04358437170288
          --> False:
            Прогноз: -20.70697423410127
        --> False:
          Прогноз: -1.5681907919679254
--> False:
  Индекс 0 <= 0.9068894675659355
  --> T

In [95]:
train_answers = predict(train_data, my_tree)
train_r2 = r2_score(train_labels, train_answers)
print(train_r2)

answers = predict(test_data, my_tree)
test_r2 = r2_score(test_labels, answers)
print(test_r2)

0.9684485721165499
0.8915084820170927
